In [1]:
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np

In [2]:
def get_text_from_genius_of(author, title):
    author = author.replace(" ", "-")
    title = title.replace(" ", "-")
    URL = "https://genius.com/" + author + "-" + title + "-lyrics"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    
    lyrics = ""
    
    lyrics_containers = soup.find_all("div", {"data-lyrics-container" : "true"})
    
    if len(lyrics_containers) == 0:
        return None
    
    for lyrics_container in lyrics_containers:
        for br in lyrics_container.find_all("br"):
            br.replace_with("\n")
        lyrics += lyrics_container.text
    
    next_brackets = lyrics.find("[")
    while next_brackets != -1:
        lyrics = lyrics[:next_brackets] + lyrics[lyrics.find("]")+1:]
        next_brackets = lyrics.find("[")
    
    lyrics = lyrics.replace("\n\n", "\n").replace("\n", " ")
    return lyrics

In [3]:
songsdata_df = pd.read_csv('batch1.csv')
songsdata_df = songsdata_df[['track', 'artist', 'genre', 'valence_tags', 'arousal_tags']]
display(songsdata_df)

,track,artist,genre,valence_tags,arousal_tags
0,Every Soul Entombed,Skáphe,black metal,3.960000,3.180000
1,Push Your Tush,Jessica Simpson,pop,6.216667,5.646667
2,Vanilla Twilight,Owl City,electronic,7.125472,4.769623
3,Fly With Me,Relaxing Dreams,electronic,6.840000,5.000000
4,The Vampire Club,Voltaire,dark cabaret,3.398125,3.245625
...,...,...,...,...,...
7495,Ride A White Swan,T. Rex,glam rock,7.940435,5.636522
7496,This Time,Life of Agony,hardcore,5.104828,5.682414
7497,"Low Level (Listening, part II)",Stars of the Lid,ambient,4.965000,4.345000
7498,A Rainbow Aims,Brightblack Morning Light,indie,4.970000,3.570000


In [4]:
def to_alfanum(txt):
    return ''.join(ch for ch in txt if ch.isalnum() or ch == ' ')


lyrics = []
for i, (_, row) in enumerate(songsdata_df.iterrows()):
    track = to_alfanum(row['track'])
    author = to_alfanum(row['artist'])
    text = get_text_from_genius_of(author, track)
    lyrics.append(text)
    if i % 1000 == 0:
        print(i)

0
1000
2000


MemoryError: 

In [5]:
print(len([t for t in lyrics if t is None]))

1282


In [8]:
songsdata_df = songsdata_df.iloc[:2361 , :]

In [9]:
songsdata_df['lyrics'] = lyrics

<ipython-input-9-fc162a9eb32a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songsdata_df['lyrics'] = lyrics


In [10]:
songsdata_df = songsdata_df.dropna()

In [11]:
display(songsdata_df)

,track,artist,genre,valence_tags,arousal_tags,lyrics
1,Push Your Tush,Jessica Simpson,pop,6.216667,5.646667,Come and get it! Woo! Do ya hear that cow bel...
2,Vanilla Twilight,Owl City,electronic,7.125472,4.769623,The stars lean down to kiss you And I lie awa...
5,Outkast,P.O.D.,hard rock,3.025000,2.725000,I'm an outcast But don't count me out I'm an ...
8,Ruby Magic,Portugal. The Man,experimental,3.652857,2.805714,Someday we will find a home (I found the way ...
9,The Day the World Went Away (Quiet),Nine Inch Nails,industrial,3.283333,2.713333,"Na na na nah Na na na na, nah Na na na nah Na ..."
...,...,...,...,...,...,...
2341,Fear,Sade,soul,6.285778,3.944000,Blue is the color of the red sky Will he—? Wi...
2342,Sunday People (Don't Bang the Drum),Flunk,electronic,4.186667,3.200000,"(The Waterboys' cover of ""Don't Bang The Drum""..."
2347,Safe and Sound,Azure Ray,indie,6.306543,3.517234,With every word I live again through the eyes ...
2348,Welfare Problems,Randy,rock,3.080000,5.870000,"Dr, dr i need a cure cause i got a problem tha..."


In [12]:
songsdata_df.to_csv('lyrics1.csv')